In [4]:
from src.io_utils import load_data, coerce_numeric

df = coerce_numeric(load_data("data/library_loans.csv"))

df.info()




ImportError: cannot import name 'coerce_numeric' from 'src.io_utils' (c:\Users\magda\OneDrive\Desktop\EDA - Bibliotek\src\io_utils.py)